<a href="https://colab.research.google.com/github/vshalisko/PythonIntro/blob/main/Unidad_2/Chatbot_oraculo_ejemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It will take exactly 4 months, 3 weeks, 2 days, 1 hour, 46 minutes and 15 seconds to learn everything you need to know to build a simple chatbot. (usuario de Reddit)

https://www.reddit.com/r/learnpython/comments/1dc80lz/making_simple_chatbot/

Chatbot inspirado en código disponible en:

https://planetachatbot.com/construyendo-chatbot-simple-desde-cero-en-python-usando-nltk/



Instalar la biblioteca gensim en el entorno virtual de Python.

Nota: Se requiere permitir reiniciar la sesiión de Colab.

In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

Cargar los componentes (bibliotecas)

In [1]:
import re
import nltk
import numpy as np

#import random

#import string

#import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

Descargar bases de datos de nltk

In [2]:
nltk.download('punkt')
#nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Cargar las palabras de español en la lista de 'stopwprds'

In [7]:
spanish_sw = set(stopwords.words('spanish'))
#print(spanish_sw)

Conectar Google Drive del usuario

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Leer el archivo ocn base de datos de frases celebres

In [4]:
path = '/content/drive/MyDrive/Colab Data/'
file = 'frases_celebres_seleccion.txt'

raw = []
with open(path + file, 'r', errors = 'ignore') as f:
  raw = f.read()
  #print(raw)

In [6]:
# convertir texto plano a lista de frases
onlysent_tokens = nltk.sent_tokenize(raw)
# reemplazar simbolos de nueva linea ocn espacios
onlysent_tokens = [sentence.replace('\n', ' ') for sentence in onlysent_tokens]

# generar una copia de texto con en lowercase
raw_lower = raw.lower()
# convertir texto a lista de palabras
word_tokens = nltk.word_tokenize(raw_lower)

print("\nNúmero de frases célebres en la base de datos:")
print(len(onlysent_tokens))

print("\nEjemplo de frases celebres:")
print(onlysent_tokens[0:3])

print("\nNúmero de palabras en labase de datos de frases célebres:")
print(len(word_tokens))



Número de frases célebres en la base de datos:
639

Ejemplo de frases celebres:
['¿Encontraría a la Maga?', 'Tan de vez en cuando uno tiene ganas de encontrarse a sí mismo.', 'A buen hambre, no hay pan duro.']

Número de palabras en labase de datos de frases célebres:
7329


Construir el modelo Word2Vec a partir de la base de datos de frases célebres

In [8]:
# Preprocesar texto para generar el modelo Word2Vec
# quitar los simbolos de punctuación y convertir palabras al registro bajo
corpus_sentences = [re.sub(r'[^\w\s]', '', sentence.lower()).split() for sentence in onlysent_tokens]
print(type(corpus_sentences))
print(len(corpus_sentences))
print(corpus_sentences[0:3])

# funcion para qliminar stopwords
def remove_stopwords(sentence_words, sw):
  filtered_words = [word for word in sentence_words if word not in sw]
  return filtered_words

# eliminar stopwords de la base de datos
filtered_sentences = []
for sentence in corpus_sentences:
  filtered_sentence = remove_stopwords(sentence, spanish_sw)
  filtered_sentences.append(filtered_sentence)

print(type(filtered_sentences))
print(len(filtered_sentences))
print(filtered_sentences[0:3])

# Entrenar el modelo Word2Vec
model = Word2Vec(min_count=1,
                     window=6,
                     vector_size=300,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=4)

model.build_vocab(filtered_sentences, progress_per=1000)
model.train(filtered_sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

# Entrenar el modelo Word2Vec (simple)
#model = Word2Vec(sentences=filtered_sentences, vector_size=100, window=5, min_count=1, workers=4)

print("\nCaracteristicas del modelo Word2Vec")
print(model)

<class 'list'>
639
[['encontraría', 'a', 'la', 'maga'], ['tan', 'de', 'vez', 'en', 'cuando', 'uno', 'tiene', 'ganas', 'de', 'encontrarse', 'a', 'sí', 'mismo'], ['a', 'buen', 'hambre', 'no', 'hay', 'pan', 'duro']]
<class 'list'>
639
[['encontraría', 'maga'], ['tan', 'vez', 'ganas', 'encontrarse', 'mismo'], ['buen', 'hambre', 'pan', 'duro']]

Caracteristicas del modelo Word2Vec
Word2Vec<vocab=1534, vector_size=300, alpha=0.03>


Preguntar al usuario por una frase

In [11]:
user_input = str(input("Introduce su frase: "))

Introduce su frase: Un hombre siempre busca la verdad, no se puede decir lo mismo sobre un caracol


Comparar la frase con la base de datos de frases celebres y buscar la frase celebre con el significado mas cercana a la frase del usuario

In [13]:

# Funcion para construir el vector de una frase
def get_sentence_vector(sentence, model):
    # separar frase en palabras
    words = sentence.split()

    # quitar simbolos de puntuacion
    words = re.sub(r'[^\w\s]', '', sentence.lower()).split()

    # determinar el vector de cada palabra en la frase en forma de lista
    word_vectors = [model.wv[word] for word in words if word in model.wv]

    # en caso que vector es vacio generar el vector de ceros y regresarlo
    if not word_vectors:
        return np.zeros(model.vector_size)

    # en caso que vector no es vacio generar un promedio y regresarlo
    return np.mean(word_vectors, axis=0)

# Get user input vector
filtered_user_input = remove_stopwords(user_input.split(), spanish_sw)
filtered_user_input_ok = " ".join(filtered_user_input)
user_input_vector = get_sentence_vector(filtered_user_input_ok, model)

print("Pregunta del ususrio: " + str(user_input))
#print("Pregunta del ususrio filtrada: " + str(filtered_user_input))
#print("Vector del modelo:")
#print(user_input_vector)

# Find the most similar sentence in the corpus
most_similar_sentence = ""
max_similarity = -1

for i, sentence in enumerate(onlysent_tokens):
    sentence_vector = get_sentence_vector(sentence, model)
    if np.linalg.norm(user_input_vector) != 0 and np.linalg.norm(sentence_vector) != 0:
        similarity = cosine_similarity([user_input_vector], [sentence_vector])[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_sentence = onlysent_tokens[i]

# La frase de mayor similitud:
#print("Primera frase")
#print(most_similar_sentence)
#print("Segunda frase")
#print(most_similar_sentence_2)


if not most_similar_sentence:
  print("No tenogo nada que responder en esta ocasión")
else:
  print("La respuesta es: " + most_similar_sentence)

Pregunta del ususrio: Un hombre siempre busca la verdad, no se puede decir lo mismo sobre un caracol
La respuesta es: El ser humano es aquel que no puede ser usado meramente como un medio, sino que debe ser siempre considerado al mismo tiempo como un fin.
